# Data Engineer Challenge

In [3]:
import pandas as pd
import json

from pandas import json_normalize

# Preparación de los datos

Debido a que el archivo de origen contenía registros malformados, fue necesario procesar el archivo línea por línea, en caso de que el archivo hubiese venido sin malformados se podría haber leido el archivo utilizando *pd.read_json*. Utilizar el método *json_normalize* además de crear el dataframe permite desempaquetar los json que están contenidos dentro de algunos registros.

In [12]:
tweets_path = r'../src/resources/farmers-protest-tweets-2021-2-4.json'

In [ ]:
tweets, bad_lines = [], []
with open(tweets_path, 'r') as file:
    for line in file:
        try:
          tweets.append(json.loads(line))
        except:
          bad_lines.append(line)

In [11]:
tweets_flat_df = json_normalize(tweets)

# Desafío

## Los top 10 tweets más retweeted.
Se utiliza la el método *nlargest* para obtener los 10 registros con más retweets, utilizando el índice se filtra el dataframe original para mostrar el contenido de los 10 tweets con mayor número de retweets

In [15]:
index_top_ten_tweets = tweets_flat_df['retweetCount'].nlargest(10).index
top_ten_tweets = tweets_flat_df.iloc[index_top_ten_tweets]
top_ten_tweets[['content','retweetCount']]  

,content,retweetCount
111329,मध्यप्रदेश में निजी व्यापारी 200 करोड़ का धान ...,7723
7645,There's a #FarmersProtest happening in Germany...,6164
89780,"disha ravi, a 21-year-old climate activist, ha...",4673
88911,Disha Ravi broke down in court room and told j...,3742
111556,Farmers are so sweet. Y’all have to see this @...,3332
64492,india is targeting young women to silence diss...,3230
108072,Bollywood has betrayed Panjab &amp; the farmer...,3182
60721,लहरों को ख़ामोश देख कर ये ना समझना कि समंदर मे...,3057
29510,"हाँ मैं जानता हूँ कि मैं शायर नहीं, और ज़ुल्म ...",3040
24160,"कलियुग है साहब , यहाँ झूठे को स्वीकार किया जा...",2622


## Los top 10 users en función a la cantidad de tweets que emitieron.
Se agrupan los tweets por usuario y se realiza un conteo por usuario, se toman los 10 valores más grandes.

In [16]:
tweets_flat_df.groupby('user.username')['content'].count().nlargest(10)

user.username
jot__b             1019
rebelpacifist       850
MaanDee08215437     830
Gurpreetd86         636
GurmVicky           597
shells_n_petals     576
preetysaini321      573
ish_kayy            515
KaurDosanjh1979     512
DigitalKisanBot     490
Name: content, dtype: int64

## Los top 10 días donde hay más tweets.
Se transforma la columna date para manejarla como fecha.  El método *pd.Grouper* permite especificar que para agrupar los datos de la columna "date" es necesario fijarse únicamente en el día, ignorando la infomación de la hora. Posteriormente se hace un conteo de los tweets y se seleccionan los 10 días con mayor número de tweets.

In [18]:
tweets_flat_df["date"] = pd.to_datetime(tweets_flat_df["date"])
tweets_flat_df.groupby(pd.Grouper(key='date', freq='D'))['content'].count().nlargest(10)

date
2021-02-12 00:00:00+00:00    12347
2021-02-13 00:00:00+00:00    11296
2021-02-17 00:00:00+00:00    11087
2021-02-16 00:00:00+00:00    10443
2021-02-14 00:00:00+00:00    10249
2021-02-18 00:00:00+00:00     9625
2021-02-15 00:00:00+00:00     9197
2021-02-20 00:00:00+00:00     8502
2021-02-23 00:00:00+00:00     8417
2021-02-19 00:00:00+00:00     8204
Name: content, dtype: int64

   ## Los top 10 hashtags más usados.
   Se utiliza una expresión regular para cachar todos las cadenas que comiencen con # que puedan (o no) terminar en espacio o punto, utiliando el método *findall* es posible obtener más de una coincidencia (hashtag) por tweet.
   
   El resultado de la primera línea devuelve un pd.Series donde cada registro contiene una lista. Se utiliza el método *explode* para convertir cada elemento de la lista en un registro y posteriormente hacer un conteo utilizando el método *value_counts* que devuelve una objeto pd.Series ordenado de forma ascendente. Se toman solo los 10 primero registros. 

In [24]:
hashtags_in_lst = tweets_flat_df["content"].str.findall(r'(#\w+)[\s|\.]?')
hashtags = hashtags_in_lst.explode()
hashtags.name = 'Hashtag'
hashtags.value_counts()[:10]

Hashtag
#FarmersProtest             112630
#ReleaseDetainedFarmers       5987
#FarmersMakeIndia             5263
#farmersprotest               5105
#MahapanchayatRevolution      4735
#RepealOnlyWayAhead           4511
#IndiaBeingSilenced           4332
#FarmersProtests              3661
#Pagdi_Sambhal_Jatta          3542
#DishaRavi                    3017
Name: count, dtype: int64

## Los top 10 emojis más usados.

## Los top 10 users más influyentes en función de lo retweeted de sus tweets.